### Running this file would scrape twitter to get content of tweets
Need to install tweepy first
- pip install tweepy

### twitter data Scraping

In [21]:
import tweepy
import pandas as pd
from configparser import ConfigParser
import time
import csv
import re

# Twitter API Configuration
config = ConfigParser(interpolation=None)
config.read('../config.ini')
BEARER_TOKEN = config['keys']['BEARER_TOKEN']  # Replace with your token
consumer_key = config['keys']['CONSUMER_KEY']
consumer_secret = config['keys']['CONSUMER_SECRET']
access_token = config['keys']['ACCESS_TOKEN']
access_token_secret = config['keys']['ACCESS_TOKEN_SECRET']

# Initialize Tweepy Client for Twitter API v2
client = tweepy.Client(
    bearer_token=BEARER_TOKEN,
    wait_on_rate_limit=True
)

# Define search parameters
search_query = '("global warming" OR "climate change") -is:retweet -is:reply -has:links lang:en'
no_of_tweets = 100  # Max per request is 100

try:
    # Fetch tweets using API v2
    tweets = client.search_recent_tweets(
        query=search_query,
        max_results=no_of_tweets,
        tweet_fields=['created_at', 'public_metrics', 'text'],
        user_fields=['name', 'username'],
        expansions=['author_id']
    )
    
    # Extract users data
    users = {u.id: u for u in tweets.includes['users']}
    
    # Prepare tweet data
    tweets_data = []
    for tweet in tweets.data:
        user = users[tweet.author_id]
        tweets_data.append({
            "User": user.name,
            "Username": user.username,
            "Date Created": tweet.created_at,
            "Likes": tweet.public_metrics['like_count'],
            "Tweet": tweet.text
        })
    
    # Create DataFrame
    tweets_df = pd.DataFrame(tweets_data)
    
except tweepy.TweepyException as e:
    print(f"API Error: {e}")
except Exception as e:
    print(f"General Error: {e}")

tweets_df

,User,Username,Date Created,Likes,Tweet
0,Glen Gustman,GGustman8083,2025-06-29 08:30:12+00:00,0,This article delves into the history of renewa...
1,lightbod.e⚡️💙,lightbod33,2025-06-29 08:24:53+00:00,0,"Dear weirdo ppl,\nDon’t be denying that your r..."
2,Keith Francis,keith_fran19338,2025-06-29 08:22:35+00:00,0,The environmental reporter was tasked with con...
3,Mariah Best,BestMariah43378,2025-06-29 08:18:43+00:00,0,Underwater assets could be the next big thing ...
4,S-ve,Sivers_Art_Den,2025-06-29 08:15:13+00:00,0,just heard a song from little spoon by patrici...
...,...,...,...,...,...
94,Kiều Tân Long,KiuTnLong185647,2025-06-29 04:00:36+00:00,0,The international agreement on climate change ...
95,athena,taxfreebakery,2025-06-29 03:58:46+00:00,0,You published that self serving book stalked m...
96,moon,sylusgon,2025-06-29 03:54:39+00:00,13,winter outfit in july... linkon climate change...
97,Mirino,Morphobia,2025-06-29 03:47:27+00:00,0,The truth that humanity is responsable for ‘cl...


In [22]:
tweets_df.to_csv('../data/tweet_details.csv', index=False, quoting=csv.QUOTE_ALL, encoding='utf-8')
print(f"Saved {len(tweets_df)} tweets to tweet_details.csv")

Saved 99 tweets to tweet_details.csv
